In [1]:
import numpy as np
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import qiskit

In [ ]:
# encoding each feature using 4 quits: equivalent to aggregating each feature over 4 time steps

# target train a comparable model on encoded data
# check for data leaks from the model

In [2]:
T=np.load('./data/Data.npy')
L=np.load('./data/Labels.npy')

In [3]:
n_w = 4 # numbers of wires def 4
dev = qml.device("default.qubit", wires= n_w)

In [4]:
n_layers = 1

# Random circuit parameters
rand_params = np.random.uniform(high= 2 * np.pi, size=(n_layers, n_w)) # def 2, n_w = 4

@qml.qnode(dev)
def circuit(phi=None):
    # Encoding of 4 classical input values
    for j in range(n_w):
        qml.RY(np.pi * phi[j], wires=j)

    # Random quantum circuit
    RandomLayers(rand_params, wires=list(range(n_w)))

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(n_w)]

In [5]:
def encode(feature,kr=4):
    out=[]
    for j in range(0,feature.shape[0]-3,4):
        q_results = circuit(
                phi=[feature[j], feature[j+1], feature[j + 2], feature[j + 3]]
            )
        out.append(q_results)
    return out

In [10]:
Enc=[]
from tqdm import tqdm
for j in tqdm(range(0,4)):
    x=T[j,:,:]
    F=[]
    for i in range(0,47):
        feature=x[:,i]
        encoded=encode(feature)
        F.append(np.stack(encoded))

    F=np.stack(F)
    F=np.transpose(F, [1, 0, 2]) 
    Enc.append(F)   
      
Enc=np.stack(Enc)
np.save('./encodeed_data.npy',Enc)    

100%|██████████| 4/4 [00:13<00:00,  3.48s/it]
